# Mismatch Order Event

## Config

In [37]:
import pandas as pd
import sys

sys.path.append('c:/Users/Jordi Cremades/Documents/Repos/central-node')

from src import query_engines

q = query_engines.QueryEngines()

## Parameters

In [9]:
DAY = "'2024-09-01'"
DAY_1 = "2024-09-01"
DAY_2 = "2024-09-02"

params = [
    {'name':'day', 'value': DAY},
    {'name':'day_1', 'value': DAY_1},
    {'name':'day_2', 'value': DAY_2},]

## Queries

In [11]:
# orders from sensitive without taking into account anything from loyalty cards
q.prepare_query(
    query_file='n_orders_sensitive.sql'
    ,params=params
    ,to_load_file=None
    ,load_from_to_load_file=None
)

n_orders_sensitive = q.query_run_starburst()
n_orders_sensitive

c:\Users/Jordi Cremades/Documents/Repos/central-node\src\query_engines.py:197: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


,creation_date,count_order_sensitive
0,2024-09-01,901568


In [12]:
# orders from sensitive taking into account loyalty cards factor
q.prepare_query(
    query_file='n_orders_sensitive_loyalty.sql'
    ,params=params
    ,to_load_file=None
    ,load_from_to_load_file=None
)

n_orders_sensitive_loyalty = q.query_run_starburst()
n_orders_sensitive_loyalty

n_orders_sensitive['count_order_sensitive'].sum()

c:\Users/Jordi Cremades/Documents/Repos/central-node\src\query_engines.py:197: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


901568

In [23]:
n_orders_sensitive_loyalty

,creation_date,custom_attributes__is_loyalty_card_added,count_order_sensitive
0,2024-09-01,None,896211
1,2024-09-01,false,4323
2,2024-09-01,true,1039


In [13]:
# orders from orders desriptors
q.prepare_query(
    query_file='order_descriptors.sql'
    ,params=params
    ,to_load_file=None
    ,load_from_to_load_file=None
)

order_descriptors = q.query_run_starburst()
order_descriptors

c:\Users/Jordi Cremades/Documents/Repos/central-node\src\query_engines.py:197: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


,p_creation_date,count_order
0,2024-09-01,937203


In [14]:
# orders from live db 
q.prepare_query(
    query_file='orders_livedb.sql'
    ,params=params
    ,to_load_file=None
    ,load_from_to_load_file=None
)

orders_livedb = q.query_run_livedb()
orders_livedb

,date(creation_time),count_order
0,2024-09-01,937205


In [21]:
# orders_loyalty_cards from 
q.prepare_query(
    query_file='orders_with_loyalty_cards.sql'
    ,params=params
    ,to_load_file=None
    ,load_from_to_load_file=None
)

orders_with_loyalty_cards = q.query_run_starburst()
orders_with_loyalty_cards

c:\Users/Jordi Cremades/Documents/Repos/central-node\src\query_engines.py:197: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


,p_creation_date,count_order
0,2024-09-01,3114


In summary for date 2024-09-01:
- 901568 orders from orders in sensitive event Order Created

- 901568 orders from orders in sensitive event Order Created when we split per LC added or not

- 937203 orders in order descriptors

- 937205 orders in live db

Focusing on the orders with loyalty cards

- 3114 orders with with loyalty card using temp_order_loyalty_cards_enriched
- 1039 ordres with loyalty card using sensitive envent Order Created and is_loyalty_card_added not null

## Compare at order level

In [52]:
# Order Level. Ordres with loyalty cards from sensitive custom event
q.prepare_query(
    query_file='n_orders_sensitive_loyalty_order_level.sql'
    ,params=params
    ,to_load_file=None
    ,load_from_to_load_file=None
)

sensitive = q.query_run_starburst()
sensitive.head()

c:\Users/Jordi Cremades/Documents/Repos/central-node\src\query_engines.py:197: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


,p_creation_date,order_id,is_loyalty_card_added
0,2024-09-01,100839362715,true
1,2024-09-01,100839322378,true
2,2024-09-01,100839207603,true
3,2024-09-01,100839296570,true
4,2024-09-01,100839253315,true


In [66]:
# Order Level. Ordres with loyalty cards from sensitive custom event
q.prepare_query(
    query_file='n_orders_delta_loyalty_order_level.sql'
    ,params=params
    ,to_load_file=None
    ,load_from_to_load_file=None
)

delta = q.query_run_starburst()
delta.head()

c:\Users/Jordi Cremades/Documents/Repos/central-node\src\query_engines.py:197: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(self.tp__read_query, conn)


,p_creation_date,order_id,order_subvertical
0,2024-09-01,100838429322,QCPartners
1,2024-09-01,100838029504,QCPartners
2,2024-09-01,100837609194,QCPartners
3,2024-09-01,100837836495,QCPartners
4,2024-09-01,100839046804,Food - Food


In [67]:
sensitive.describe()

,p_creation_date,order_id,is_loyalty_card_added
count,1039,1039,1039
unique,1,1039,1
top,2024-09-01,100839362715,true
freq,1039,1,1039


In [68]:
delta.describe()

,p_creation_date,order_id,order_subvertical
count,3114,3114,3114
unique,1,3114,2
top,2024-09-01,100838429322,QCPartners
freq,3114,1,2343


In [69]:
# merged
tmp = pd.merge(delta,sensitive, on='order_id', how='left', suffixes=('_delta','_sensitive'))
tmp

,p_creation_date_delta,order_id,order_subvertical,p_creation_date_sensitive,is_loyalty_card_added
0,2024-09-01,100838429322,QCPartners,NaN,NaN
1,2024-09-01,100838029504,QCPartners,2024-09-01,true
2,2024-09-01,100837609194,QCPartners,NaN,NaN
3,2024-09-01,100837836495,QCPartners,NaN,NaN
4,2024-09-01,100839046804,Food - Food,NaN,NaN
...,...,...,...,...,...
3109,2024-09-01,100838662797,QCPartners,NaN,NaN
3110,2024-09-01,100837661690,QCPartners,NaN,NaN
3111,2024-09-01,100838327799,QCPartners,NaN,NaN
3112,2024-09-01,100837420141,QCPartners,2024-09-01,true


In [70]:
# keep only the ones that were not found
cond = tmp['p_creation_date_sensitive'].isnull()

print(tmp['order_id'][cond].nunique(), 'Orders that appear as with loyalty in the delta but not in sensitive')

2087 Orders that appear as with loyalty in the delta but not in sensitive


In [72]:
# all these orders do not appear in the custom event with loyalty card added. They appear as order_id, but the field
# of loyalty card added does not.
tmp[cond]

,p_creation_date_delta,order_id,order_subvertical,p_creation_date_sensitive,is_loyalty_card_added
0,2024-09-01,100838429322,QCPartners,NaN,NaN
2,2024-09-01,100837609194,QCPartners,NaN,NaN
3,2024-09-01,100837836495,QCPartners,NaN,NaN
4,2024-09-01,100839046804,Food - Food,NaN,NaN
5,2024-09-01,100837392202,QCPartners,NaN,NaN
...,...,...,...,...,...
3108,2024-09-01,100837616977,QCPartners,NaN,NaN
3109,2024-09-01,100838662797,QCPartners,NaN,NaN
3110,2024-09-01,100837661690,QCPartners,NaN,NaN
3111,2024-09-01,100838327799,QCPartners,NaN,NaN
